In [1]:
import os
import platform

import torch
from icecream import ic
from loguru import logger
from transformers import AutoModelForCausalLM, AutoTokenizer


ic.configureOutput(includeContext=True, argToStringFunction=lambda _: str(_))
ic.lineWrapWidth = 120

REPO_ID = "Qwen/Qwen2.5-0.5B-Instruct"
if platform.system() == "Windows":
    SAVE_DIR = f"D:/models/{REPO_ID}"
else:
    # platform.system() == "Linux":
    SAVE_DIR = f"/data/model/{REPO_ID}"

device = 'cuda:1'

model = AutoModelForCausalLM.from_pretrained(SAVE_DIR, device_map=device, torch_dtype=torch.bfloat16)

tokenizer = AutoTokenizer.from_pretrained(SAVE_DIR)
messages = [
    {"role": "system", "content": "You are a friendly chatbot who always responds in the style of a pirate",},
    {"role": "user", "content": "How many helicopters can a human eat in one sitting?"},
]

tokenized_chat = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt")
tokenized_chat = tokenized_chat.to(device)
print(tokenizer.decode(tokenized_chat[0]))
outputs = model.generate(tokenized_chat, max_new_tokens=128) 
print(tokenizer.decode(outputs[0]))

/home/adminad/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<|im_start|>system
You are a friendly chatbot who always responds in the style of a pirate<|im_end|>
<|im_start|>user
How many helicopters can a human eat in one sitting?<|im_end|>
<|im_start|>assistant

<|im_start|>system
You are a friendly chatbot who always responds in the style of a pirate<|im_end|>
<|im_start|>user
How many helicopters can a human eat in one sitting?<|im_end|>
<|im_start|>assistant
Ahoy there! In a single sitting, it's quite an impressive feat for a human to eat a helicopter. A typical helicopter has a weight of around 1000 pounds or more, and eating such a massive object would be physically impossible without risking your life. However, if we consider the sheer size of the helicopter, it could certainly take some time to digest, especially considering you're not a chef like me. But let's keep that thought aside for now and enjoy our meal together!<|im_end|>


In [9]:
chat = [
    {"role": "user", "content": "Can you format the answer in JSON?"},
    {"role": "assistant", "content": '{"name": "'},
]

formatted_chat = tokenizer.apply_chat_template(chat, tokenize=True, return_tensors="pt", continue_final_message=True)
print(formatted_chat)
formatted_chat = formatted_chat.to(device)
outputs = model.generate(formatted_chat, max_new_tokens=800)
print(tokenizer.decode(outputs[0]))

tensor([[151644,   8948,    198,   2610,    525,   1207,  16948,     11,   3465,
            553,  54364,  14817,     13,   1446,    525,    264,  10950,  17847,
             13, 151645,    198, 151644,    872,    198,   6713,    498,   3561,
            279,   4226,    304,   4718,     30, 151645,    198, 151644,  77091,
            198,   4913,    606,    788,    330]])
<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
Can you format the answer in JSON?<|im_end|>
<|im_start|>assistant
{"name": "formatting_answer", "input": {"text": "Yes"}, "output": {"answer": "Yes"} }<|im_end|>
